<a href="https://colab.research.google.com/github/wenqiglantz/nvidia-sec-finetuning/blob/main/embedding-finetuning/finetune_embedding_nvidia_sec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fine-tune Embedding Model for NVIDIA SEC Filing Analysis

## Installation and Configuration

In [ ]:
!pip install llama_index==0.8.23.post1 pypdf sentence-transformers

In [29]:
import json
import openai
import os

from llama_index import SimpleDirectoryReader
from llama_index.node_parser import SimpleNodeParser
from llama_index.schema import MetadataMode
from llama_index import (
    VectorStoreIndex,
    SimpleDirectoryReader,
    ServiceContext,
    Response
)

In [30]:
!curl https://d18rn0p25nwr6d.cloudfront.net/CIK-0001045810/4e9abe7b-fdc7-4cd2-8487-dc3a99f30e98.pdf --output nvidia-sec-10k-2022.pdf

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1541k  100 1541k    0     0  3475k      0 --:--:-- --:--:-- --:--:-- 3478k


## Generate dataset

### Load corpus

Load corpus function, can be called for either training dataset (first 90 pages) or eval dataset (the rest of the 79 pages).  for_training flag defaults to false.

In [31]:
def load_corpus(docs, for_training=False, verbose=False):
    parser = SimpleNodeParser.from_defaults()
    if for_training:
        nodes = parser.get_nodes_from_documents(docs[:90], show_progress=verbose)
    else:
        nodes = parser.get_nodes_from_documents(docs[91:], show_progress=verbose)

    if verbose:
        print(f'Parsed {len(nodes)} nodes')

    return nodes

In [ ]:
SEC_FILE = ['nvidia-sec-10k-2022.pdf']

print(f"Loading files {SEC_FILE}")

reader = SimpleDirectoryReader(input_files=SEC_FILE)
docs = reader.load_data()
print(f'Loaded {len(docs)} docs')

train_nodes = load_corpus(docs, for_training=True, verbose=True)
val_nodes = load_corpus(docs, for_training=False, verbose=True)

Loading files ['nvidia-sec-10k-2022.pdf']
Loaded 169 docs


[nltk_data] Downloading package punkt to /tmp/llama_index...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Parsing documents into nodes:   0%|          | 0/90 [00:00<?, ?it/s]

Parsed 97 nodes


Parsing documents into nodes:   0%|          | 0/78 [00:00<?, ?it/s]

Parsed 91 nodes


### Generate synthetic queries

In [ ]:
from llama_index.finetuning import (
    generate_qa_embedding_pairs,
    EmbeddingQAFinetuneDataset,
)
from llama_index.llms import OpenAI

Now let's generate our dataset for training and evaluation.  Notice we are not passing llm (`gpt-3.5-turbo-0613`) here, only the OpenAI API key, because LlamaIndex's default model is `gpt-3.5-turbo-0613`, it defaults to it if no llm is defined and OpenAI API key is provided.

In [ ]:
os.environ["OPENAI_API_KEY"] = "sk-################"
openai.api_key = os.environ["OPENAI_API_KEY"]

train_dataset = generate_qa_embedding_pairs(train_nodes)
val_dataset = generate_qa_embedding_pairs(val_nodes)

train_dataset.save_json("train_dataset.json")
val_dataset.save_json("val_dataset.json")

100%|██████████| 91/91 [03:48<00:00,  2.52s/it]


In [ ]:
train_dataset = EmbeddingQAFinetuneDataset.from_json("train_dataset.json")
val_dataset = EmbeddingQAFinetuneDataset.from_json("val_dataset.json")

## Fine-tune embedding model

In [ ]:
from llama_index.finetuning import SentenceTransformersFinetuneEngine

In [ ]:
finetune_engine = SentenceTransformersFinetuneEngine(
    train_dataset,
    model_id="BAAI/bge-small-en",
    model_output_path="test_model",
    val_dataset=val_dataset,
)

In [ ]:
finetune_engine.finetune()

Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

Iteration:   0%|          | 0/20 [00:00<?, ?it/s]

Iteration:   0%|          | 0/20 [00:00<?, ?it/s]

In [ ]:
embed_model = finetune_engine.get_finetuned_model()

In [ ]:
embed_model

LangchainEmbedding(model_name='test_model', embed_batch_size=10, callback_manager=<llama_index.callbacks.base.CallbackManager object at 0x7f253af1e590>)

## Evaluate fine-tuned model

In [ ]:
from llama_index.embeddings import OpenAIEmbedding
from llama_index import ServiceContext, VectorStoreIndex
from llama_index.schema import TextNode
from tqdm.notebook import tqdm
import pandas as pd

In [ ]:
def evaluate(
    dataset,
    embed_model,
    top_k=5,
    verbose=False,
):
    corpus = dataset.corpus
    queries = dataset.queries
    relevant_docs = dataset.relevant_docs

    service_context = ServiceContext.from_defaults(embed_model=embed_model)
    nodes = [TextNode(id_=id_, text=text) for id_, text in corpus.items()]
    index = VectorStoreIndex(nodes, service_context=service_context, show_progress=True)
    retriever = index.as_retriever(similarity_top_k=top_k)

    eval_results = []
    for query_id, query in tqdm(queries.items()):
        retrieved_nodes = retriever.retrieve(query)
        retrieved_ids = [node.node.node_id for node in retrieved_nodes]
        expected_id = relevant_docs[query_id][0]
        is_hit = expected_id in retrieved_ids  # assume 1 relevant doc

        eval_result = {
            "is_hit": is_hit,
            "retrieved": retrieved_ids,
            "expected": expected_id,
            "query": query_id,
        }
        eval_results.append(eval_result)
    return eval_results

In [ ]:
from sentence_transformers.evaluation import InformationRetrievalEvaluator
from sentence_transformers import SentenceTransformer


def evaluate_st(
    dataset,
    model_id,
    name,
):
    corpus = dataset.corpus
    queries = dataset.queries
    relevant_docs = dataset.relevant_docs

    evaluator = InformationRetrievalEvaluator(queries, corpus, relevant_docs, name=name)
    model = SentenceTransformer(model_id)
    return evaluator(model, output_path="results/")

### Run Evals

#### OpenAI

In [ ]:
ada = OpenAIEmbedding()
ada_val_results = evaluate(val_dataset, ada)

Generating embeddings:   0%|          | 0/91 [00:00<?, ?it/s]

  0%|          | 0/182 [00:00<?, ?it/s]

In [ ]:
df_ada = pd.DataFrame(ada_val_results)

In [ ]:
hit_rate_ada = df_ada['is_hit'].mean()
hit_rate_ada

0.9065934065934066

#### BAAI/bge-small-en

In [ ]:
bge = "local:BAAI/bge-small-en"
bge_val_results = evaluate(val_dataset, bge)

Generating embeddings:   0%|          | 0/91 [00:00<?, ?it/s]

  0%|          | 0/182 [00:00<?, ?it/s]

In [ ]:
df_bge = pd.DataFrame(bge_val_results)

In [ ]:
hit_rate_bge = df_bge['is_hit'].mean()
hit_rate_bge

0.8516483516483516

In [ ]:
evaluate_st(val_dataset, "BAAI/bge-small-en", name='bge')

0.6914052642120058

If you run into error at this step, be sure to check if you have "results" folder created at your project root, if not, make sure to create it and re-run this step.

#### Fine-tuned model

In [ ]:
finetuned = "local:test_model"
val_results_finetuned = evaluate(val_dataset, finetuned)

Generating embeddings:   0%|          | 0/91 [00:00<?, ?it/s]

  0%|          | 0/182 [00:00<?, ?it/s]

In [ ]:
df_finetuned = pd.DataFrame(val_results_finetuned)

In [ ]:
hit_rate_finetuned = df_finetuned['is_hit'].mean()
hit_rate_finetuned

0.8626373626373627

In [ ]:
evaluate_st(val_dataset, "test_model", name='finetuned')

0.7315810324641175

### Summary of results

#### Hit rate

In [32]:
df_ada['model'] = 'ada'
df_bge['model'] = 'bge'
df_finetuned['model'] = 'fine_tuned'

In [33]:
df_all = pd.concat([df_ada, df_bge, df_finetuned])
df_all.groupby('model').mean('is_hit')

,is_hit
model,
ada,0.906593
bge,0.851648
fine_tuned,0.862637


import numpy as np
from google.colab import autoviz

def value_plot(df, y, figscale=1):
  from matplotlib import pyplot as plt
  df[y].plot(kind='line', figsize=(8 * figscale, 4 * figscale), title=y)
  plt.gca().spines[['top', 'right']].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = value_plot(_df_0, *['is_hit'], **{})
chart

import numpy as np
from google.colab import autoviz

def histogram(df, colname, num_bins=20, figscale=1):
  from matplotlib import pyplot as plt
  df[colname].plot(kind='hist', bins=num_bins, title=colname, figsize=(8*figscale, 4*figscale))
  plt.gca().spines[['top', 'right',]].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = histogram(_df_1, *['is_hit'], **{})
chart

#### InformationRetrievalEvaluator

In [35]:
df_st_bge = pd.read_csv('results/Information-Retrieval_evaluation_bge_results.csv')
df_st_finetuned = pd.read_csv('results/Information-Retrieval_evaluation_finetuned_results.csv')

df_st_bge['model'] = 'bge'
df_st_finetuned['model'] = 'fine_tuned'
df_st_all = pd.concat([df_st_bge, df_st_finetuned])
df_st_all = df_st_all.set_index('model')
df_st_all

,epoch,steps,cos_sim-Accuracy@1,cos_sim-Accuracy@3,cos_sim-Accuracy@5,cos_sim-Accuracy@10,cos_sim-Precision@1,cos_sim-Recall@1,cos_sim-Precision@3,cos_sim-Recall@3,...,dot_score-Recall@1,dot_score-Precision@3,dot_score-Recall@3,dot_score-Precision@5,dot_score-Recall@5,dot_score-Precision@10,dot_score-Recall@10,dot_score-MRR@10,dot_score-NDCG@10,dot_score-MAP@100
model,,,,,,,,,,,,,,,,,,,,,
bge,-1,-1,0.576923,0.758242,0.835165,0.901099,0.576923,0.576923,0.252747,0.758242,...,0.582418,0.254579,0.763736,0.164835,0.824176,0.090110,0.901099,0.687227,0.738769,0.691405
fine_tuned,-1,-1,0.631868,0.780220,0.862637,0.939560,0.631868,0.631868,0.260073,0.780220,...,0.631868,0.258242,0.774725,0.174725,0.873626,0.093407,0.934066,0.728595,0.778090,0.731502
